# Iterative simulation of generating discharge documentation

This notebook simulates the performance of iteratively (daily) generating a new discharge document based on the document from yesterday and the daily status of today

In [ ]:
import logging
import os
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

from discharge_docs.prompts.prompt import (
    load_prompts,
    load_template_prompt,
)
from discharge_docs.prompts.prompt_builder import PromptBuilder

In [ ]:
# initialise Azure
load_dotenv("../.env")
deployment_name = "aiva-gpt"
TEMPERATURE = 0.2

client = AzureOpenAI(
    api_version="2024-02-01",
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT", ""),
)

In [ ]:
logging.basicConfig(level=logging.INFO)

## Load data

In [ ]:
df_metavision = pd.read_parquet(
    Path().cwd().parent / "data" / "processed" / "metavision_new_data.parquet"
)
patient_1_NICU = df_metavision[df_metavision["enc_id"] == 107]
patient_1_IC = df_metavision[df_metavision["enc_id"] == 48]

df_HIX = pd.read_parquet(
    Path().cwd().parent / "data" / "processed" / "HiX_data.parquet"
)
patient_1_CAR = df_HIX[df_HIX["enc_id"] == 1012]

## Simulate patient from NICU

In [ ]:
user_prompt, system_prompt = load_prompts(iterative=True)
template_prompt = load_template_prompt("NICU")

In [ ]:
print(system_prompt)

In [ ]:
print(user_prompt)

In [ ]:
print(template_prompt)

In [ ]:
prompt_builder = PromptBuilder(
    temperature=TEMPERATURE,
    deployment_name=deployment_name,
    client=client,
)

In [ ]:
discharge_letters = prompt_builder.iterative_simulation(
    patient_1_NICU, system_prompt, user_prompt, template_prompt
)

In [ ]:
discharge_letters[4]

In [ ]:
from pprint import pprint

pprint(discharge_letters[-1])

In [ ]:
print(
    patient_1_NICU.loc[
        patient_1_NICU["description"].str.contains("ontslagbrief"), "value"
    ].values[-1]
)

## Simulate patient from IC

In [ ]:
template_prompt = load_template_prompt("IC")

In [ ]:
discharge_letters = prompt_builder.iterative_simulation(
    patient_1_IC, system_prompt, user_prompt, template_prompt
)

In [ ]:
pprint(discharge_letters[-1])

In [ ]:
pprint(
    patient_1_IC.loc[
        patient_1_IC["description"].str.contains("ontslagbrief"), "value"
    ].values[-1]
)

## Simulatie Cardio

In [ ]:
template_prompt = load_template_prompt("CAR")

In [ ]:
discharge_letters = prompt_builder.iterative_simulation(
    patient_1_CAR, system_prompt, user_prompt, template_prompt
)

In [ ]:
pprint(discharge_letters[-1])

In [ ]:
pprint(
    patient_1_CAR.loc[
        patient_1_CAR["description"].str.contains("Ontslagbrief"), "value"
    ].values[-1]
)